In [1]:
import torch 
import torch.nn as nn 
from torch.optim import Adam
from scr.model import KyrgyzLetterCNN
import matplotlib.pyplot as plt
from scr.dataset import CustomKyrgyzDataset
from tqdm import tqdm 
import torchvision.transforms.v2 as tfs 
from torch.utils.data import DataLoader

# Загружаем данные

In [2]:
train_transform = tfs.Compose([
    tfs.RandomRotation(10),        
    tfs.RandomAffine(0, translate=(0.1, 0.1)),       # Случайное смещение
    tfs.ToTensor(),
    tfs.Normalize((0.5,), (0.5,))
])

val_transfrom = tfs.Compose([
    tfs.ToTensor(),
    tfs.Normalize((0.5,), (0.5,))
])

C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [9]:
train_dataset = CustomKyrgyzDataset('../data/train.csv', train = True, transform=train_transform)
val_dataset = CustomKyrgyzDataset('../data/train.csv', train = True, transform=val_transfrom)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Модель

In [10]:
model = KyrgyzLetterCNN()
device = torch.device('cpu')
model.to(device)

loss_func = nn.CrossEntropyLoss()   # loss 
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.001)

best_val_acc = 0.0

In [11]:
for epoch in range(1, 11):
    model.train()
    train_loss = 0.0
    correct = 0 
    total = 0
    print(f'Эпоха {epoch}')
    for images, labels in tqdm(train_loader, desc= 'Обучение'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
    train_acc = correct / total
    print(f'Эпоха {epoch}, Потери: {train_loss}, Точность: {train_acc}')
    
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Валидация'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
            
    val_acc = correct / total
    print(f'Точность на валидации: {val_acc}')
    
        # Сохраняем лучшую модель
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "../models/kyrgyzletters_model.pt")
        print("Модель сохранена!")

Эпоха 1


Обучение: 100%|██████████| 1254/1254 [02:46<00:00,  7.55it/s]


Эпоха 1, Потери: 2941.4856778383255, Точность: 0.2927330981262389


Валидация: 100%|██████████| 1254/1254 [01:09<00:00, 17.92it/s]


Точность на валидации: 0.2927330981262389
Модель сохранена!
Эпоха 2


...

Точность на валидации: 0.883896625235311
Модель сохранена!
Эпоха 10


Обучение: 100%|██████████| 1254/1254 [03:25<00:00,  6.09it/s]


Эпоха 10, Потери: 447.2346366047859, Точность: 0.8928727263660504


Валидация: 100%|██████████| 1254/1254 [01:33<00:00, 13.35it/s]

Точность на валидации: 0.8928727263660504
Модель сохранена!
